# Topic 3: Recommender System

## Business Objective/Problem
- Tiki là một hệ sinh thái thương mại “all in one”, trong
đó có tiki.vn, là một website thương mại điện tử đứng
top 2 của Việt Nam, top 6 khu vực Đông Nam Á.
- Trên trang này đã triển khai nhiều tiện ích hỗ trợ nâng
cao trải nghiệm người dùng và họ muốn xây dựng
nhiều tiện ích hơn nữa.
- Giả sử công ty này chưa triển khai Recommender
System và bạn được yêu cầu triển khai hệ thống này,
bạn sẽ làm gì?
Đ

## Các kiến thức/ kỹ năng cần để giải quyết
vấn đề này:
- Hiểu vấn đề
- Import các thư viện cần thiết và hiểu cách sử dụng
- Đọc dữ liệu (dữ liệu project này được cung cấp)
- Thực hiện EDA cơ bản (sử dụng Pandas Profiling
Report)
- Tiền xử lý dữ liệu: làm sạch, tạo tính năng mới, lựa
chọn tính năng cần thiết…

## Bước 1: Business Understanding
- Dựa vào yêu cầu nói trên => xác định vấn đề:
  - Chưa có hệ thống Recommendation System
  - => Mục tiêu/ vấn đề: Xây dựng Recommendation System
  cho một hoặc một số nhóm hàng hóa trên tiki.vn giúp đề
  xuất và gợi ý cho người dùng/ khách hàng. => Xây dựng
  các mô hình đề xuất:
    - Content-based filtering
    - Collaborative filtering

Bước 2: Data Understanding/ Acquire
- Từ mục tiêu/ vấn đề đã xác định: xem xét các dữ
liệu cần thiết:
  - Dữ liệu được cung cấp sẵn gồm có các tập tin:
  ProductRaw.csv, ReviewRaw.csv chứa thông tin sản phẩm,
  review và rating cho các sản phẩm thuộc các nhóm hàng
  hóa như Mobile_Tablet, TV_Audio, Laptop, Camera,
  Accessory

# Bước 3: Data preparation/ Prepare

# 5. ALS

In [ ]:
! pip install pandas-profiling==2.7.1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install surprise
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

     |████████████████████████████████| 252 kB 8.3 MB/s 
     |████████████████████████████████| 58 kB 2.5 MB/s 
     |████████████████████████████████| 675 kB 17.9 MB/s 
     |████████████████████████████████| 4.7 MB 43.8 MB/s 
     |████████████████████████████████| 812 kB 44.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.5 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=070e6c7b9166c554efb2ff82186568aada05022e6aa4cb13d7c30edbab7d644d
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=5ec4fcf0b80316fa4fa4ff29c1e3db85ac25f6c8da7c9504d9aece460f6238e3
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built htmlmin imagehash
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3'

/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import FloatType
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sc = SparkContext()

In [ ]:
spark = SparkSession.builder.appName('ALS').getOrCreate()

In [ ]:
# Loads data.
products_df = spark.read.csv("Data/Products.csv", 
                     inferSchema= True,
                     header=True,
                     sep=',',
                     multiLine=True,
                     quote="\"",
                     escape="\"")

In [ ]:
products_df.show(5)

+--------+--------------------+--------------------+------+-------+----------+--------+--------------------+--------------------+--------------------+
| item_id|                name|         description|rating|  price|list_price|   brand|               group|                 url|               image|
+--------+--------------------+--------------------+------+-------+----------+--------+--------------------+--------------------+--------------------+
|48102821|Tai nghe Bluetoot...|THÔNG TIN CHI TIẾ...|   4.0|  77000|    300000|     OEM|Thiết Bị Số - Phụ...|https://tai-nghe-...|https://salt.tiki...|
|52333193|Tai nghe bluetoot...|THÔNG TIN CHI TIẾ...|   4.5| 132000|    750000|     OEM|Thiết Bị Số - Phụ...|https://tai-nghe-...|https://salt.tiki...|
|  299461|Chuột Không Dây L...|THÔNG TIN CHI TIẾ...|   4.8| 299000|    399000|Logitech|Thiết Bị Số - Phụ...|https://chuot-kho...|https://salt.tiki...|
|57440329|Loa Bluetooth 5.0...|THÔNG TIN CHI TIẾ...|   4.7| 149000|    350000|   Acome|Thiết B

In [ ]:
reviews = pd.read_csv('Data/Reviews.csv',lineterminator='\n')
reviews.head()                  

,customer_id,product_id,name,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361090 entries, 0 to 361089
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  361090 non-null  int64 
 1   product_id   361090 non-null  int64 
 2   name         360662 non-null  object
 3   rating       361090 non-null  int64 
 4   title        361062 non-null  object
 5   content      165053 non-null  object
dtypes: int64(3), object(3)
memory usage: 16.5+ MB


In [ ]:
schema = StructType([StructField( 'customer_id', IntegerType(), True),StructField( 'product_id', IntegerType(), True),StructField('name', StringType(), True),StructField( 'rating', IntegerType(), True),StructField( 'title', StringType(), True),StructField('content', StringType(), True)])
reviews = spark.createDataFrame(data=reviews, schema= schema)     

In [ ]:
reviews.show(3)

+-----------+----------+----------------+------+------------------+--------------------+
|customer_id|product_id|            name|rating|             title|             content|
+-----------+----------+----------------+------+------------------+--------------------+
|     709310|  10001012|Lân Nguyễn Hoàng|     3|Ko dùng đc thẻ nhớ|Lúcđầu quên thông...|
|   10701688|  10001012|Nguyễn Khánh Hòa|     5|   Cực kì hài lòng|Tiki giao hàng nh...|
|   11763074|  10001012| Toàn Phạm Khánh|     5|   Cực kì hài lòng|chất lượng camera...|
+-----------+----------+----------------+------+------------------+--------------------+
only showing top 3 rows



In [ ]:
reviews.columns

['customer_id', 'product_id', 'name', 'rating', 'title', 'content']

- Đây là bài toán Filtering 
chỉ lấy dữ liệu 3 cột để phân tích


In [ ]:
# Chỉ lấy dữ liệu ở 3 cột ['product_id','rating','customer_id']
data = reviews.select(['product_id','rating','customer_id'])
data.show(3)

+----------+------+-----------+
|product_id|rating|customer_id|
+----------+------+-----------+
|  10001012|     3|     709310|
|  10001012|     5|   10701688|
|  10001012|     5|   11763074|
+----------+------+-----------+
only showing top 3 rows



In [ ]:
data.describe().show()

+-------+--------------------+------------------+-----------------+
|summary|          product_id|            rating|      customer_id|
+-------+--------------------+------------------+-----------------+
|  count|              361090|            361090|           361090|
|   mean|2.4406563573685233E7| 4.475136392589105|9179677.248489296|
| stddev| 2.378465728570825E7|1.0166716679634726|6307971.007714791|
|    min|               54665|                 1|               10|
|    max|            81964004|                 5|         21013443|
+-------+--------------------+------------------+-----------------+



In [ ]:
data.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [ ]:
# Distinct users and products
users = reviews.select('customer_id').distinct().count()
products = reviews.select('product_id').distinct().count()
numerator = reviews.count()

In [ ]:
display(numerator, users, products)

361090

251149

4214

In [ ]:
# Number of raings matrix could contain if no empty cells
denominator = users * products
denominator

1058341886

In [ ]:
# Calculating spaarsity
sparsity = 1 - (numerator*1.0/denominator)
print('Sparsity: '), sparsity

Sparsity: 


(None, 0.999658815355627)

In [ ]:
# Check Null - Nan
data.select([count(when(isnan(c)|isnull(c),c)).alias (c) for c in data.columns]).show()

+----------+------+-----------+
|product_id|rating|customer_id|
+----------+------+-----------+
|         0|     0|          0|
+----------+------+-----------+



In [ ]:
# Chia tập dữ liệu thành train và test
(training, test) = data.randomSplit([0.8,0.2])

# Model


In [ ]:
data.describe().show()

+-------+--------------------+------------------+-----------------+
|summary|          product_id|            rating|      customer_id|
+-------+--------------------+------------------+-----------------+
|  count|              361090|            361090|           361090|
|   mean|2.4406563573685233E7| 4.475136392589105|9179677.248489296|
| stddev| 2.378465728570825E7|1.0166716679634726|6307971.007714791|
|    min|               54665|                 1|               10|
|    max|            81964004|                 5|         21013443|
+-------+--------------------+------------------+-----------------+



In [ ]:
data.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [ ]:
from time import time
t0 = time()
als = ALS(maxIter=5, regParam=0.09,rank=25, userCol='customer_id', itemCol= 'product_id',ratingCol='rating', coldStartStrategy='drop', nonnegative= True)
model = als.fit(training)
t = time() - t0
t

24.23385453224182

# Đánh giá kết quả

In [ ]:
# Evalute the model by computing the RMSE on the test data
predictions = model.transform(test)

In [ ]:
predictions.show(5)

+----------+------+-----------+----------+
|product_id|rating|customer_id|prediction|
+----------+------+-----------+----------+
|    605223|     5|    8591725| 2.5919783|
|   2069769|     5|    7005222| 2.2801661|
|   2069769|     1|   14596766| 3.8261404|
|   2069769|     5|   13931579| 5.2268744|
|   2086067|     5|   11296538| 5.2807603|
+----------+------+-----------+----------+
only showing top 5 rows



In [ ]:
evaluator = RegressionEvaluator(metricName='rmse',
                               labelCol='rating',
                               predictionCol='prediction')

In [ ]:
rmse = evaluator.evaluate(predictions)
print('Root_mean_squar_error:', rmse)

Root_mean_squar_error: 2.116825497610228


rmse = 2.12 > std = 1.16
Thử tìm kiếm một model khác để giảm rmse xuống nếu có thể

---



In [ ]:
t0 = time()
als_t = ALS(maxIter=25, regParam=0.1,rank= 25,  userCol='customer_id', itemCol= 'product_id',ratingCol='rating', coldStartStrategy='drop', nonnegative= True)
model_t = als_t.fit(training)
time() - t0

63.359699964523315

In [ ]:
predictions_t = model_t.transform(test)

In [ ]:
rmse_t = evaluator.evaluate(predictions_t)
print('Root_mean_squar_error:', rmse_t)

Root_mean_squar_error: 1.1986054346100823


- rmse giảm xuống = 1.19 ~ std = 1.16 => chọn model_t

In [ ]:
# Đưa ra đề xuất cho tất cả các user
# Get 10 recommendations
user_recs = model_t.recommendForAllUsers(10)

In [ ]:
user_recs.printSchema()

root
 |-- customer_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- product_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [ ]:
user_recs.show(3, False)

+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|customer_id|recommendations                                                                                                                                                                                                                |
+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|471        |[[66251373, 5.277201], [10690707, 5.0774937], [71293311, 5.071312], [72323554, 4.996207], [50692303, 4.9147205], [55897, 4.8897676], [75983219, 4.8622026], [491431, 4.8546195], [742272, 4.847637], [50560429, 4.847374]]     |
|833        |[[71293311, 5.092226], [44227610, 4

In [ ]:
for user in user_recs.head(3):
    print(user)

Row(customer_id=471, recommendations=[Row(product_id=66251373, rating=5.277201175689697), Row(product_id=10690707, rating=5.077493667602539), Row(product_id=71293311, rating=5.071311950683594), Row(product_id=72323554, rating=4.996207237243652), Row(product_id=50692303, rating=4.91472053527832), Row(product_id=55897, rating=4.889767646789551), Row(product_id=75983219, rating=4.8622026443481445), Row(product_id=491431, rating=4.85461950302124), Row(product_id=742272, rating=4.847637176513672), Row(product_id=50560429, rating=4.847373962402344)])
Row(customer_id=833, recommendations=[Row(product_id=71293311, rating=5.092226028442383), Row(product_id=44227610, rating=4.891177177429199), Row(product_id=66251373, rating=4.857330322265625), Row(product_id=72323554, rating=4.8535661697387695), Row(product_id=48520298, rating=4.8459858894348145), Row(product_id=76732229, rating=4.84362268447876), Row(product_id=68527587, rating=4.835090160369873), Row(product_id=54023388, rating=4.808348178863

In [ ]:
recs = user_recs.toPandas()
recs.head()

,customer_id,recommendations
0,471,"[(66251373, 5.277201175689697), (10690707, 5.0..."
1,833,"[(71293311, 5.092226028442383), (44227610, 4.8..."
2,7993,"[(25965194, 4.887197971343994), (65439708, 4.8..."
3,9427,"[(71293311, 1.0456827878952026), (40351876, 1...."
4,17420,"[(71293311, 5.492371559143066), (66251373, 5.4..."


In [ ]:
nrecs = recs['recommendations'].apply(pd.Series).merge(recs, right_index= True, left_index= True).drop(['recommendations'],axis = 1).melt(id_vars = ['customer_id'], value_name = 'recommendations').drop('variable', axis = 1).dropna()

In [ ]:
nrecs.head()

,customer_id,recommendations
0,471,"(66251373, 5.277201175689697)"
1,833,"(71293311, 5.092226028442383)"
2,7993,"(25965194, 4.887197971343994)"
3,9427,"(71293311, 1.0456827878952026)"
4,17420,"(71293311, 5.492371559143066)"


In [ ]:
nrecs = nrecs.sort_values('customer_id')
nrecs.head()

,customer_id,recommendations
340164,10,"(416911, 3.911929130554199)"
2031252,10,"(71938164, 3.795532464981079)"
1185708,10,"(71293311, 3.843348503112793)"
1608480,10,"(24558526, 3.8100805282592773)"
551550,10,"(48520298, 3.8965165615081787)"


In [ ]:
nrecs = pd.concat([nrecs['recommendations'].apply(pd.Series), nrecs['customer_id']],axis=1)

In [ ]:
nrecs.columns = ['product_id', 'rating', 'customer_id']

In [ ]:
nrecs.head()

,product_id,rating,customer_id
340164,416911.0,3.911929,10
2031252,71938164.0,3.795532,10
1185708,71293311.0,3.843349,10
1608480,24558526.0,3.810081,10
551550,48520298.0,3.896517,10


In [ ]:
md = data.toPandas()

In [ ]:
md

,product_id,rating,customer_id
0,10001012,3,709310
1,10001012,5,10701688
2,10001012,5,11763074
3,10001012,5,9909549
4,10001012,5,1827148
...,...,...,...
361085,9996258,5,16820553
361086,9996258,5,5888640
361087,9996258,5,12246414
361088,9996258,5,14309619


In [ ]:
nrecs = nrecs.sort_values('customer_id')
nrecs.reset_index(drop=True, inplace= True)
new = nrecs[['customer_id','product_id','rating']]
new['recommendations'] = list(zip(new.product_id, new.rating))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
new.head()

,customer_id,product_id,rating,recommendations
0,10,416911.0,3.911929,"(416911.0, 3.911929130554199)"
1,10,71938164.0,3.795532,"(71938164.0, 3.795532464981079)"
2,10,71293311.0,3.843349,"(71293311.0, 3.843348503112793)"
3,10,24558526.0,3.810081,"(24558526.0, 3.8100805282592773)"
4,10,48520298.0,3.896517,"(48520298.0, 3.8965165615081787)"


In [ ]:
res = new[['customer_id','recommendations']]

In [ ]:
res_new = res['recommendations'].groupby([res['customer_id']]).apply(list).reset_index()
res_new.head()

,customer_id,recommendations
0,10,"[(416911.0, 3.911929130554199), (71938164.0, 3..."
1,27,"[(19395453.0, 5.037433624267578), (74227763.0,..."
2,28,"[(76732229.0, 4.828741550445557), (40351876.0,..."
3,36,"[(10690707.0, 5.307761192321777), (74274442.0,..."
4,50,"[(66251373.0, 4.043923854827881), (48520298.0,..."


In [ ]:
res_new.columns

Index(['customer_id', 'recommendations'], dtype='object')

In [ ]:
final_recs = spark.createDataFrame(res_new, schema=['customer_id', 'recommendations']) # Lưu lại dữ liệu này

In [ ]:
final_recs.count()

211386

In [ ]:
final_recs.show(5)

+-----------+--------------------+
|customer_id|     recommendations|
+-----------+--------------------+
|         10|[[416911.0, 3.911...|
|         27|[[1.9395453E7, 5....|
|         28|[[7.6732229E7, 4....|
|         36|[[1.0690707E7, 5....|
|         50|[[6.6251373E7, 4....|
+-----------+--------------------+
only showing top 5 rows



# Make recommendations to some users:

In [ ]:
def recs(userID, score):
  print('Recomend for userID = ',userID,'\n')
  result = final_recs.filter(final_recs['customer_id'] == userID)
  result = result.select(result['customer_id'], explode(result['recommendations']))

  result = result.withColumn('product_id',result.col.getField('_1'))\
              .withColumn('rating', result.col.getField('_2').cast(FloatType()))
  products_id_name = products_df.withColumn('product_id', products_df.item_id).select(['name','product_id'])
  result = result.join(products_id_name, on= ['product_id'], how='left_outer')
  result.filter(result['rating'] > score ).show(truncate = False)
  return result

In [ ]:
userID = 10
score = 3
result = recs(userID, score)

Recomend for userID =  10 

+-----------+-----------+---------------------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id |customer_id|col                              |rating   |name                                                                                                                                                                                  |
+-----------+-----------+---------------------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|416911.0   |10         |[416911.0, 3.911929130554199]    |3.9119291|TP-Link  TL-WN823N - USB Wifi chuẩn N tốc độ 300Mbps - Hàng Chính Hãng                                                                          

In [ ]:
userID = 27
score = 4.5
result = recs(userID, score)

Recomend for userID =  27 

+-----------+-----------+---------------------------------+---------+-----------------------------------------------------------------------------------------------+
|product_id |customer_id|col                              |rating   |name                                                                                           |
+-----------+-----------+---------------------------------+---------+-----------------------------------------------------------------------------------------------+
|1.9395453E7|27         |[1.9395453E7, 5.037433624267578] |5.0374336|MÁY RỬA CHÉN BOSCH HMH.SMS63L08EA -HÀNG CHÍNH HÃNG                                          |
|7.4227763E7|27         |[7.4227763E7, 5.099873065948486] |5.099873 |Smart Tivi Casper Full HD 43 inch 43FG5200                                                     |
|7.6310232E7|27         |[7.6310232E7, 5.077657222747803] |5.077657 |Đồng Hồ Thông Minh Theo Dõi Vận Động Theo Dõi Sức Khỏe Huami Amazfit GTR 

In [ ]:
# Lưu model
model_t.save('Results/ALS/Model_ALS')

In [ ]:
# Lưu dữ liệu
final_recs.write.parquet('Results/ALS/ALS', mode = 'overwrite')